<a href="https://colab.research.google.com/github/sukruthisantosh/SFT/blob/main/colab-sft-project-1/SFT_colab_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers datasets torch trl accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 104.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.2/376.2 kB 34.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [3]:
# Import libraries
import torch
import pandas as pd
from datasets import load_dataset
from transformers import TrainingArguments, AutoTokenizer, AutoModelForCausalLM
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM, SFTConfig

In [4]:
def load_model_and_tokenizer(model_name, use_gpu):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)
    if use_gpu:
        model = model.to('cuda')
    return model, tokenizer

def generate_responses(model, tokenizer, input_texts):
    inputs = tokenizer(input_texts, return_tensors='pt', padding=True, truncation=True)
    if next(model.parameters()).is_cuda:
        inputs = {key: value.to('cuda') for key, value in inputs.items()}
    outputs = model.generate(**inputs)
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)

def test_model_with_questions(model, tokenizer, questions, title="Model Output"):
    print(f"=== {title} ===")
    responses = generate_responses(model, tokenizer, questions)
    for question, response in zip(questions, responses):
        print(f"Q: {question}\nA: {response}\n")

def display_dataset(dataset):
    df = pd.DataFrame(dataset)
    print(df.head())  # Display the first few rows of the dataset

In [8]:
# Load base model & test on simple questions
USE_GPU = torch.cuda.is_available()
print(USE_GPU)

questions = [
    "Give me an 1-sentence introduction of LLM.",
    "Calculate 1+1-1",
    "What's the difference between thread and process?"
]

model, tokenizer = load_model_and_tokenizer("Qwen/Qwen3-0.6B-Base", USE_GPU)

test_model_with_questions(model, tokenizer, questions,
                          title="Base Model (Before SFT) Output")


True


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


=== Base Model (Before SFT) Output ===
Q: Give me an 1-sentence introduction of LLM.
A: Give me an 1-sentence introduction of LLM. A large language model (LLM) is a type of artificial intelligence that can understand and generate human-like text based on its training data.

Q: Calculate 1+1-1
A: Calculate 1+1-1 the answer is 1. Can you explain why this is the case?

The expression \(1 + 1 - 1\) can be evaluated step by step. First, add 1 and 1, which equals 2. Then, subtract 1 from 2, resulting in 1. Therefore, the answer is 1. This is because the order of operations (PEMDAS/BODMAS) dictates that you perform addition before subtraction. In this case, adding 1 and 1 gives 2, and then subtracting 1 from 2 results in 1.

Q: What's the difference between thread and process?
A: What's the difference between thread and process? the difference between thread and process?
Thread and process are two fundamental concepts in operating systems, each serving distinct purposes and operating in diffe

In [13]:

train_dataset = load_dataset("banghua/DL-SFT-Dataset")["train"]
if not USE_GPU:
    train_dataset=train_dataset.select(range(100))

display_dataset(train_dataset)

                                            messages
0  [{'content': '- The left child should have a v...
1  [{'content': 'To pass three levels must be the...
2  [{'content': 'Can you translate the text mater...
3  [{'content': 'Complete feed for exotic fishes ...
4  [{'content': 'Write a funny limerick about a p...


In [14]:
sft_config = SFTConfig(
    learning_rate=8e-5, # Learning rate for training.
    num_train_epochs=1, #  Set the number of epochs to train the model.
    per_device_train_batch_size=1, # Batch size for each device (e.g., GPU) during training.
    gradient_accumulation_steps=8, # Number of steps before performing a backward/update pass to accumulate gradients.
    gradient_checkpointing=False, # Enable gradient checkpointing to reduce memory usage during training at the cost of slower training speed.
    logging_steps=2,  # Frequency of logging training progress (log every 2 steps).
    bf16=False,  # Disable bf16 for CPU training
    fp16=False,  # Disable fp16 for CPU training
)

sft_trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=train_dataset,
    processing_class=tokenizer,
)
sft_trainer.train()

Tokenizing train dataset:   0%|          | 0/2961 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/2961 [00:00<?, ? examples/s]

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sukruthisantosh (sukruthisantosh-imperial-college-london) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
2,4.069800
4,3.133900
6,2.668900
8,2.374000
10,2.159800
12,2.392400
14,2.380900
16,2.240700
18,2.226600
20,2.093200


wandb: WARNING The get_url method is deprecated and will be removed in a future release. Please use `run.url` instead.


TrainOutput(global_step=371, training_loss=2.0388934021047507, metrics={'train_runtime': 1149.6408, 'train_samples_per_second': 2.576, 'train_steps_per_second': 0.323, 'total_flos': 1255369248866304.0, 'train_loss': 2.0388934021047507})

In [19]:
# Testing training results on small model and small dataset
# if not USE_GPU:  # move model to CPU when GPU isn’t requested
#     sft_trainer.model.to("cpu")

model, tokenizer = load_model_and_tokenizer("Qwen/Qwen3-0.6B-Base", USE_GPU)

test_model_with_questions(model, tokenizer, questions,
                          title="Base Model (After SFT) Output")

del model, tokenizer

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


=== Base Model (After SFT) Output ===
Q: Give me an 1-sentence introduction of LLM.
A: Give me an 1-sentence introduction of LLM. A large language model (LLM) is a type of artificial intelligence that can understand and generate human-like text based on its training data.

Q: Calculate 1+1-1
A: Calculate 1+1-1 the answer is 1. Can you explain why this is the case?

The expression \(1 + 1 - 1\) can be evaluated step by step. First, add 1 and 1, which equals 2. Then, subtract 1 from 2, resulting in 1. Therefore, the answer is 1. This is because the order of operations (PEMDAS/BODMAS) dictates that you perform addition before subtraction. In this case, adding 1 and 1 gives 2, and then subtracting 1 from 2 results in 1.

Q: What's the difference between thread and process?
A: What's the difference between thread and process? the difference between thread and process?
Thread and process are two fundamental concepts in operating systems, each serving distinct purposes and operating in differ